In [1]:
import gzip
import random

In [2]:
from collections import defaultdict

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [3]:
data=list(readGz("assignment1/train.json.gz"))
train_data = data[:100000]
valid_data = data[100000:200000]

In [4]:
user_item = defaultdict(list)
users = set()
items =set()
for l in data:
    user,item = l['reviewerID'],l['itemID']
    users.add(user)
    items.add(item)
    user_item[user].append(item)

In [5]:
negative_pair = []
users = list(users)
items = list(items)
count = 0
while count < 100000:
    userrandom = random.choice(users)
    itemrandom = random.choice(items)
    pair = ((userrandom,itemrandom))
    if itemrandom in user_item[userrandom]:
            continue
    else:
        negative_pair.append(pair)
    count+=1

In [6]:
user_valid = [x['reviewerID'] for x in valid_data]
item_valid = [x['itemID'] for x in valid_data]

In [7]:
positive_pair = list(zip(user_valid,item_valid))

In [8]:
whole_pair = positive_pair + negative_pair

In [9]:
businessCount = defaultdict(int)
totalPurchases = 0

for l in data:
  user,business = l['reviewerID'],l['itemID']
  businessCount[business] += 1
  totalPurchases += 1

mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalPurchases*0.401: break

In [10]:
user_category = defaultdict(list)
business_category = defaultdict(list)
users = set()
businesses = set()
for l in data:
    user,business,category= l['reviewerID'],l['itemID'],l['categories']
    users.add(user)
    businesses.add(business)
    user_category[user].append(category)
    business_category[business].append(category)

In [11]:
for i in users:
    y = []
    for j in user_category[i]:
        for h in j:
            y.append(tuple(h))
    user_category[i] = set(y)

In [12]:
for i in businesses:
    y = []
    for j in business_category[i]:
        for h in j:
            y.append(tuple(h))
    business_category[i] = set(y)

In [13]:
def find1(x,y):
    sim = []
    for a in user_item[x]:
        count = 0
        thing = []
        for i in business_category[a]:
            thing.append(i)
            for j in business_category[y]:
                thing.append(j)
                if i == j:
                    count+=1
        if count >0:
            sim.append(count/len(set(thing)))
        else:
            sim.append(0)
        sim.sort(reverse = True)
    if len(sim)>1:
        if sim[0] + sim[1] >= 0.44:
            return True
        else:
            return False
    elif len(sim) == 1:
        if sim[0] >= 0.0925:
            return True
        else:
            return False
    else:
        return False

In [13]:
def find(x,y):
    count = 0
    thing = []
    for i in user_category[x]:
        thing.append(i)
        for j in business_category[y]:
            thing.append(j)
            if i == j:
                count += 1
    if count > 0:
        if count/len(set(thing))>=0.0925:
            return True
        else:
            return False
    else:
        return False

In [14]:
predictions = open("assignment1/predictions_Purchase.csv", 'w')
for l in open("assignment1/pairs_Purchase.txt"):
  if l.startswith("reviewerID"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  h = i in return1
  if find1(u,i) | h:
    predictions.write(u + '-' + i + ",1\n")
  else:
    predictions.write(u + '-' + i + ",0\n")
  

predictions.close()